### Script for generating prediction CSV files for Dev and Test sets

**GA-Ensembling weights and individual model predictions already available**

In [46]:
import pandas as pd
import numpy as np

In [47]:
# Reading CSV from link
def read_csv_from_link(url):
    path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
    df = pd.read_csv(path,delimiter="\t",error_bad_lines=False, header=None)
    return df

In [48]:
# Loading All Data
# tamil_train = read_csv_from_link('https://drive.google.com/file/d/15auwrFAlq52JJ61u7eSfnhT9rZtI5sjk/view?usp=sharing')
# tamil_dev = read_csv_from_link('https://drive.google.com/file/d/1Jme-Oftjm7OgfMNLKQs1mO_cnsQmznRI/view?usp=sharing')
# malayalam_train = read_csv_from_link('https://drive.google.com/file/d/13JCCr-IjZK7uhbLXeufptr_AxvsKinVl/view?usp=sharing')
# malayalam_dev = read_csv_from_link('https://drive.google.com/file/d/1J0msLpLoM6gmXkjC6DFeQ8CG_rrLvjnM/view?usp=sharing')
# kannada_train = read_csv_from_link('https://drive.google.com/file/d/1BFYF05rx-DK9Eb5hgoIgd6EcB8zOI-zu/view?usp=sharing')
kannada_dev = read_csv_from_link('https://drive.google.com/file/d/1V077dMQvscqpUmcWTcFHqRa_vTy-bQ4H/view?usp=sharing')

In [49]:
# Test
tamil_test = read_csv_from_link('https://drive.google.com/file/d/10RHrqXvIKMdnvN_tVJa_FAm41zaeC8WN/view?usp=sharing')
malayalam_test = read_csv_from_link('https://drive.google.com/file/d/1zx1wCC9A-Pp80mzbqixb52WlWQQ7ATyJ/view?usp=sharing')
kannada_test = read_csv_from_link('https://drive.google.com/file/d/1Px2CvIkLP_xaNhz_fCofW-7GGBCnSYsa/view?usp=sharing')

b'Skipping line 778: expected 2 fields, saw 3\n'


In [50]:
split = 'test'

In [51]:
# Uncomment If making predictions for Dev-Set
# kannada_test = kannada_dev
# split = 'dev'

In [52]:
# Mal Preprocess
kannada_test = kannada_test.iloc[:, 0:2]
kannada_test = kannada_test.rename(columns={0: "text", 1: "GT_label"})

### Load Model

In [6]:
import torch

In [7]:
print("GPU Nos: {}".format(torch.cuda.device_count()))
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

# Change Device - CPU/GPU-0/GPU-1
torch.cuda.set_device(0)
device = 'cuda'
device = device if torch.cuda.is_available() else 'cpu'

GPU Nos: 2
Tesla P100-PCIE-12GB
Tesla P100-PCIE-16GB


In [ ]:
# Load Weights dictionary

In [7]:
import pickle
saved_model_filename = 'GA_v1_kannada'

with open("../../dev_preds/weights_" + saved_model_filename + ".pickle", 'rb') as handle:
    mw = pickle.load(handle)

In [8]:
mw

{'XLMroberta_custom_pretrained_kannada.pth': (2.4575588857477992,
  'xlm-roberta-base'),
 'Distilbert_m_base_cased_kannada_weighted.pth': (1.8119959704070476,
  'distilbert-base-multilingual-cased'),
 'Indic_bert_kannada_weighted.pth': (0.462110920903549,
  'ai4bharat/indic-bert'),
 'Distilbert_m_base_cased_kannada.pth': (1.6341476131375288,
  'distilbert-base-multilingual-cased'),
 'XLMroberta_custom_pretrained_kannada_weighted.pth': (2.65716323320213,
  'xlm-roberta-base'),
 'MURIL_cased_temp_kannada.pth': (1.2740285448075563,
  'simran-kh/muril-cased-temp'),
 'XLMroberta_large_kannada_weighted.pth': (4.016614405001666,
  'xlm-roberta-large'),
 'Mbert_base_cased_kannada.pth': (3.302975964858024,
  'bert-base-multilingual-cased'),
 'Mbert_base_cased_kannada_weighted.pth': (2.124259887478466,
  'bert-base-multilingual-cased'),
 'XLMroberta_large_kannada.pth': (0.7028579920165603, 'xlm-roberta-large'),
 'Indic_bert_kannada.pth': (0.7455879960046707, 'ai4bharat/indic-bert'),
 'MURIL_case

In [17]:
all_probs = []
import os

In [18]:
label_mapping = {
    'Not_offensive': 0, 
    'not-Kannada': 1, 
    'Offensive_Targeted_Insult_Other': 2, 
    'Offensive_Targeted_Insult_Group': 3, 
    'Offensive_Untargetede': 4, 
    'Offensive_Targeted_Insult_Individual': 5
}

In [13]:
# Loading Model
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
from tqdm.notebook import tqdm

for index in tqdm(mw.keys()):
    model_name = index
    pretrained_key = mw[index][1]
    try:
        test_preds = np.load("../../test_probs/"+model_name+".npy")
        all_probs.append(test_preds)
        continue
    except:
        pass

    if pretrained_key == 'distilbert-base-multilingual-cased':
        tokenizer = BertTokenizer.from_pretrained(pretrained_key)
        model = BertForSequenceClassification.from_pretrained(pretrained_key, num_labels=6)
    else:
        tokenizer = AutoTokenizer.from_pretrained(pretrained_key)
        model = AutoModelForSequenceClassification.from_pretrained(pretrained_key, num_labels=6)

    state_dict = torch.load(os.path.join('../../finetuned_models/', model_name))
    model.load_state_dict(state_dict)
    model.eval()

    # Collecting Text and Labels
    test_batch_sentences = list(kannada_test['text'])

    # Convert to Tensor
    test_encodings = tokenizer(test_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt")

    # Dataset
    from torch.utils.data import Dataset

    class kannada_Offensive_Dataset(Dataset):
        def __init__(self, encodings):
            self.encodings = encodings

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(1) # Label is irrelevant, since we need predictions
            return item

        def __len__(self):
            return len(self.encodings["input_ids"])

    # Defining Datasets
    test_dataset = kannada_Offensive_Dataset(test_encodings)

    ### Run Models-1
    from torch.utils.data import DataLoader
    from tqdm.notebook import tqdm
    from sklearn.metrics import classification_report, f1_score

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)

    # Dataloaders
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    test_preds = []
    with torch.set_grad_enabled(False):
        for batch in tqdm(test_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            for logits in outputs[1].cpu().numpy():
                test_preds.append(np.exp(logits)/np.sum(np.exp(logits)))
    
    np.save("../../test_probs/"+model_name+".npy", test_preds)
    all_probs.append(test_preds)
    del model
    torch.cuda.empty_cache()

In [65]:
all_probs = []
if split == 'test':
    for k in mw.keys():
        all_probs.append(np.load("../../test_probs/"+k+".npy")[:-1])
else:
    for k in mw.keys():
        all_probs.append(np.load("../../model_prediction_probs/preds_"+k+".npy"))

In [66]:
mw

{'XLMroberta_custom_pretrained_kannada.pth': (2.4575588857477992,
  'xlm-roberta-base'),
 'Distilbert_m_base_cased_kannada_weighted.pth': (1.8119959704070476,
  'distilbert-base-multilingual-cased'),
 'Indic_bert_kannada_weighted.pth': (0.462110920903549,
  'ai4bharat/indic-bert'),
 'Distilbert_m_base_cased_kannada.pth': (1.6341476131375288,
  'distilbert-base-multilingual-cased'),
 'XLMroberta_custom_pretrained_kannada_weighted.pth': (2.65716323320213,
  'xlm-roberta-base'),
 'MURIL_cased_temp_kannada.pth': (1.2740285448075563,
  'simran-kh/muril-cased-temp'),
 'XLMroberta_large_kannada_weighted.pth': (4.016614405001666,
  'xlm-roberta-large'),
 'Mbert_base_cased_kannada.pth': (3.302975964858024,
  'bert-base-multilingual-cased'),
 'Mbert_base_cased_kannada_weighted.pth': (2.124259887478466,
  'bert-base-multilingual-cased'),
 'XLMroberta_large_kannada.pth': (0.7028579920165603, 'xlm-roberta-large'),
 'Indic_bert_kannada.pth': (0.7455879960046707, 'ai4bharat/indic-bert'),
 'MURIL_case

In [67]:
weights = [mw[index][0] for index in mw]

In [68]:
inverse_label_mapping = {v: k for k, v in label_mapping.items()}

In [69]:
X = weights
all_dev_preds = np.array(all_probs)

softmax_w = np.exp(X)/np.sum(np.exp(X))
weighted_all_dev_preds = np.array([sw*dpreds for sw, dpreds in zip(softmax_w, all_dev_preds)])
weighted_dev_preds = np.sum(weighted_all_dev_preds, axis = 0)
final_dev_preds = np.argmax(weighted_dev_preds, axis = 1)

final_dev_pred_labels = [inverse_label_mapping[int(k)] for k in final_dev_preds]
saved_model_filename = 'test_GA_v1_kannada'
kannada_test['labels'] = final_dev_pred_labels

In [70]:
kannada_test.index.name = 'id'

In [71]:
kannada_test

,text,GT_label,labels
id,,,
0,ಜೈ ಮೋರಿ ಜೈ ರೋಗಿ ಇಸ್ಲಾಂ ಸಾಯಿಸಿ ಇದೆ ಇವರ ಗುರಿ,Offensive_Targeted_Insult_Group,Not_offensive
1,Anna nim e vedio nodinu mathe chaina apps use ...,Offensive_Targeted_Insult_Group,Not_offensive
2,Super anna yenu thappila yela sari agi hellidi...,Not_offensive,Not_offensive
3,Super sir super super super super,not-Kannada,not-Kannada
4,AGT-KELU (KANNADA RAP 2019),not-Kannada,not-Kannada
...,...,...,...
772,Super movie 🤩,not-Kannada,not-Kannada
773,Startup start maadalu capacity growth maadalu ...,Not_offensive,Not_offensive
774,Rashmika idanna nodi thi*a urkimbeku🤣🤣🤣🤣🤣,Offensive_Targeted_Insult_Individual,Offensive_Targeted_Insult_Individual


In [72]:
from sklearn.metrics import classification_report

print(classification_report(kannada_test['GT_label'], kannada_test['labels']))

                                      precision    recall  f1-score   support

                       Not_offensive       0.79      0.85      0.82       426
     Offensive_Targeted_Insult_Group       0.50      0.30      0.37        44
Offensive_Targeted_Insult_Individual       0.72      0.68      0.70        75
     Offensive_Targeted_Insult_Other       0.50      0.07      0.12        14
               Offensive_Untargetede       0.50      0.06      0.11        33
                         not-Kannada       0.73      0.87      0.80       185

                            accuracy                           0.76       777
                           macro avg       0.62      0.47      0.49       777
                        weighted avg       0.74      0.76      0.73       777



In [43]:
# Saving predictions
kannada_test.to_csv("../../test_preds/" + saved_model_filename + ".tsv", sep = '\t')
np.save("../../test_probs/"+saved_model_filename+".npy", weighted_dev_preds)

#### Performance of each model separately

In [34]:
import numpy as np
import pickle
saved_model_filename = 'GA_v1_kannada'

with open("../../dev_preds/weights_" + saved_model_filename + ".pickle", 'rb') as handle:
    mw = pickle.load(handle)

In [35]:
all_model_names = mw.keys()

In [36]:
all_test_preds = []
for model_name in all_model_names:
    all_test_preds.append(np.load("../../test_probs/"+model_name+".npy")[:-1])

In [38]:
from sklearn.metrics import classification_report

In [43]:
label_mapping = {
    'Not_offensive': 0, 
    'not-Kannada': 1, 
    'Offensive_Targeted_Insult_Other': 2, 
    'Offensive_Targeted_Insult_Group': 3, 
    'Offensive_Untargetede': 4, 
    'Offensive_Targeted_Insult_Individual': 5
}

# Collecting Text and Labels
test_batch_sentences = list(kannada_test['text'])
test_batch_labels =  [label_mapping[x] for x in kannada_test['label']]

for test_preds, mn in zip(all_test_preds, all_model_names):
    final_test_preds = np.argmax(test_preds, axis = 1)
    y_true = test_batch_labels
    y_pred = final_test_preds
    target_names = label_mapping.keys()
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(mn)
    print(report)

XLMroberta_custom_pretrained_kannada.pth
                                      precision    recall  f1-score   support

                       Not_offensive       0.82      0.81      0.82       426
                         not-Kannada       0.75      0.89      0.81       185
     Offensive_Targeted_Insult_Other       0.00      0.00      0.00        14
     Offensive_Targeted_Insult_Group       0.43      0.43      0.43        44
               Offensive_Untargetede       0.14      0.06      0.09        33
Offensive_Targeted_Insult_Individual       0.62      0.68      0.65        75

                            accuracy                           0.75       777
                           macro avg       0.46      0.48      0.47       777
                        weighted avg       0.72      0.75      0.73       777

Distilbert_m_base_cased_kannada_weighted.pth
                                      precision    recall  f1-score   support

                       Not_offensive       0.76     

/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
